#  深度分子特征

在分子数据上进行机器学习的最重要步骤之一是将数据转化为适合的可应用于机器学习算法的形式。这个过程统称为 "特征化"，包括将分子变成某种向量或张量。有许多不同的方法可以做到这一点，而选择什么样的特征化往往取决于手头的问题。我们已经看到了两种方法：分子指纹和用于图形卷积的 `ConvMol` 对象。在本教程中，我们将看一下其他一些方法。

## Colab

本教程和目录中的其余部分都是在 Google colab 中完成。如果您想在 colab 中打开此笔记本，您可以点击以下链接。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BioGavin/deepchem/blob/master/examples/tutorials-zh/Going_Deeper_on_Molecular_Featurizations.ipynb)



In [ ]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

## 特征器

在 DeepChem 中，一个分子（或任何其他类型的输入）的特征化方法是由一个 `Featurizer` 对象定义的。有三种不同的方式使用特征器。

1. 在使用 MoleculeNet 加载器函数时，你只需传递要使用的特征化方法的名称。我们在以前的教程中看到过这样的例子，比如 `featurizer='ECFP'` 或 `featurizer='GraphConv'`。

2. 你也可以创建一个 Featurizer 并直接应用于分子。比如说：

In [1]:
import deepchem as dc

featurizer = dc.feat.CircularFingerprint()
print(featurizer(['CC', 'CCC', 'CCO']))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


3. 当用 DataLoader 框架创建一个新的数据集时，你可以指定一个用于处理数据的 Featurizer 。我们将在未来的教程中看到这一点。

在本教程中，我们使用丙烷（CH<sub>3</sub>CH<sub>2</sub>CH<sub>3</sub>，SMILES字符串表示为 `'CCC'` ）作为一个运行实例。许多特征化方法使用了分子的构象，使用 `deepchem.utils.conformers` 中的 `ConformerGenerator` 类可以生成构象。

### RDKitDescriptors

`RDKitDescriptors` 通过使用 RDKit 来计算描述符列表的值，从而对分子进行描述。这些是基本的物理和化学属性：分子量、极性表面积、氢键供体和受体的数量等等。这对于预测取决于这些高级属性而不是详细的分子结构的事物是最有用的。

featurizer 内部的是一组可以使用 `RDKitDescriptors.allowedDescriptors` 获取的分子描述符。featurizer 使用 `rdkit.Chem.Descriptors.descList` 中的描述符，检查它们是否在允许的描述符列表中，并计算出分子的描述符值。

让我们打印丙烷的前十个描述符的值。

In [2]:
rdkit_featurizer = dc.feat.RDKitDescriptors()
features = rdkit_featurizer(['CCC'])[0]
for feature, descriptor in zip(features[:10], rdkit_featurizer.descriptors):
    print(descriptor, feature)

MaxEStateIndex 2.125
MinEStateIndex 1.25
MaxAbsEStateIndex 2.125
MinAbsEStateIndex 1.25
qed 0.3854706587740357
MolWt 44.097
HeavyAtomMolWt 36.033
ExactMolWt 44.062600255999996
NumValenceElectrons 20.0
NumRadicalElectrons 0.0


当然，还有许多比这更多的描述符。

In [3]:
print('The number of descriptors present is: ', len(features))

The number of descriptors present is:  200


### WeaveFeaturizer 和 MolGraphConvFeaturizer

我们之前研究了图卷积，它使用 `ConvMolFeaturizer` 将分子转换成 `ConvMol` 对象。图卷积是一大类架构的一个特例，它将分子表示为图形。它们以类似的方式工作，但在细节上有所不同。例如，它们可以将数据向量与原子、连接它们的键或两者联系起来，它们可以使用各种技术从上一层的数据向量中计算出新的数据向量，并在最后计算出分子水平的属性。

DeepChem 支持许多不同的基于图形的模型。它们中有一些需要以稍微不同的方式对分子进行特征化。正因为如此，还有 "WeaveFeaturizer" 和 "MolGraphConvFeaturizer" 特征器。它们各自将分子转换成不同类型的 Python 对象，供特定模型使用。当使用任何基于图形的模型时，只需查看文档，看看你需要使用哪种 featurizer。

### CoulombMatrix

到目前为止，我们所看到的所有模型都只考虑了分子的内在属性：组成分子的原子列表和连接它们的键。在处理柔性分子时，你可能还想考虑分子可能采取的不同构象。例如，当一个药物分子与一个蛋白质结合时，结合的强度取决于原子对之间的特定相互作用。为了预测结合强度，你可能要考虑各种可能的构象，并在预测时使用一个考虑到这些构象的模型。

库仑矩阵是分子构象的一个流行特征。回顾一下，两个电荷之间的静电库仑作用与 $q_1 q_2/r$ 成正比，其中 $q_1$ 和 $q_2$ 是电荷，$r$ 是它们之间的距离。对于有 N 个原子的分子，库仑矩阵是一个$N \times N$ 的矩阵，其中每个元素都给出了两个原子之间静电作用的强度，包含关于原子上的电荷和它们之间的距离的信息。关于所使用的函数形式的更多信息可以在[这里](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.108.058301)找到。

为了应用这个 featurizer，我们首先需要一组分子的构象。我们可以使用 `ConformerGenerator` 类来完成这个任务。它需要一个 RDKit 分子，生成一组能量最小化的构象，并对这组构象进行修剪，只包括那些彼此之间有显著差异的构象。让我们试着对丙烷运行它。

In [4]:
from rdkit import Chem

generator = dc.utils.ConformerGenerator(max_conformers=5)
propane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCC'))
print("Number of available conformers for propane: ", len(propane_mol.GetConformers()))

Number of available conformers for propane:  1


它只发现了一个单一的构象体，这并不令人惊讶，因为丙烷是一个非常小的分子，几乎没有任何灵活性。让我们尝试添加另一个碳。

In [5]:
butane_mol = generator.generate_conformers(Chem.MolFromSmiles('CCCC'))
print("Number of available conformers for butane: ", len(butane_mol.GetConformers()))

Number of available conformers for butane:  3


现在我们可以为我们的分子创建一个库伦矩阵。

In [6]:
coulomb_mat = dc.feat.CoulombMatrix(max_atoms=20)
features = coulomb_mat(propane_mol)
print(features)

[[[36.8581052  12.48684429  7.5619687   2.85945193  2.85804514
    2.85804556  1.4674015   1.46740144  0.91279491  1.14239698
    1.14239675  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [12.48684429 36.8581052  12.48684388  1.46551218  1.45850736
    1.45850732  2.85689525  2.85689538  1.4655122   1.4585072
    1.4585072   0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 7.5619687  12.48684388 36.8581052   0.9127949   1.14239695
    1.14239692  1.46740146  1.46740145  2.85945178  2.85804504
    2.85804493  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 2.85945193  1.46551218  0.9127949   0.5         0.29325367
    0.29325369  0.21256978  0.21256978  0.12268391  0.13960187
    0.13960185  0.          0.          0.          0.
    0.          0.          0.          0.          0.        ]
  [ 2.85804514  1.458

/Users/peastman/workspace/deepchem/deepchem/feat/molecule_featurizers/coulomb_matrices.py:141: RuntimeWarning: divide by zero encountered in true_divide
  m = np.outer(z, z) / d


注意，很多元素都是0。为了在一个批次中组合多个分子，我们需要所有库仑矩阵的大小相同，即使分子的原子数不同。我们指定了 `max_atoms=20`，所以返回的矩阵大小为（20，20）。 该分子只有11个原子，所以只有11乘11的子矩阵是非零的。

### CoulombMatrixEig

库仑矩阵的一个重要特点是，它们对分子旋转和平移是不变的，因为原子间距离和原子序数不会改变，尊重这样的对称性使学习更容易。旋转一个分子不会改变其物理特性，如果特征化确实改变了，那么模型就会被迫学习旋转并不重要，但如果特征化是不变的，那么模型就会自动得到这个属性。

库仑矩阵在另一个重要的对称性下不是不变的：原子指数的排列组合。一个分子的物理属性并不取决于我们称之为 "原子1" 的原子，但库仑矩阵却取决于它。为了解决这个问题，引入了 `CoulumbMatrixEig` 特征化工具，它使用库伦矩阵的特征值谱，对原子指数的随机排列是不变的。这种特征化的缺点是它所包含的信息要少得多（ $N$ 个特征值，而不是一个 $N/times N$ 的矩阵），所以模型所能学到的东西将受到更大的限制。

`CoulombMatrixEig` 继承自 `CoulombMatrix`，首先通过计算分子的不同构象的库仑矩阵，然后计算每个库仑矩阵的特征值，来实现分子的特征化。然后，这些特征值被填充，以考虑到不同分子的原子数量的变化。


In [7]:
coulomb_mat_eig = dc.feat.CoulombMatrixEig(max_atoms=20)
features = coulomb_mat_eig(propane_mol)
print(features)

[[60.07620303 29.62963149 22.75497781  0.5713786   0.28781332  0.28548338
   0.27558187  0.18163794  0.17460999  0.17059719  0.16640098  0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]]


# 恭喜！是时候加入社区了！

恭喜您完成本教程笔记本！如果您喜欢本教程并希望继续使用 DeepChem，我们鼓励您完成本系列的其余教程。您还可以通过以下方式帮助 DeepChem 社区：

## 在 [GitHub](https://github.com/deepchem/deepchem) 上为 DeepChem 点亮小星星
这有助于大家建立对 DeepChem 项目和我们正在尝试构建的开源药物发现工具的共识。

## 加入 DeepChem Gitter
DeepChem [Gitter](https://gitter.im/deepchem/Lobby) 聚集了许多对生命科学深度学习感兴趣的科学家、开发人员和爱好者，欢迎加入！